In [24]:
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/issues/providers/Microsoft.MachineLearningServices/workspaces/issues',
 'name': 'issues',
 'location': 'northcentralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '617d4e28-19eb-47e6-9710-14e2cfc8f1e4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-01-10T13:33:11.1916827+00:00',
 'containerRegistry': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/issues/providers/Microsoft.ContainerRegistry/registries/issuesa24d4dd6',
 'keyVault': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/issues/providers/microsoft.keyvault/vaults/issues3539678377',
 'applicationInsights': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/issues/providers/microsoft.insights/components/issues5093552984',
 'identityPrincipalId': '75c4c007-89ad-4262-8739-998c0656c95b',
 'identityTenantId': '72f988bf-86f1-41

In [25]:
from azureml.core import Experiment
experiment = Experiment(ws, 'issues')

In [26]:
#ws.get_default_datastore().upload('data', 'data')
ds = ws.get_default_datastore()
#ds.upload('data', 'data')

In [29]:
from azureml.train.estimator import Estimator

params = {
    '--data_path': ws.get_default_datastore().path('data'), 
    '--analyze': '',
    '--load_open': '',
    '--load_closed': '',
}

est = Estimator(source_directory='.', 
                compute_target=ws.compute_targets['cpu'], 
                entry_script='azureml-issues.py', 
                script_params=params, pip_packages=['azure-devops','pandas'])



In [30]:
run = experiment.submit(est)
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [31]:
run.get_details()

{'runId': 'issues_1578962492_a1ac4f24',
 'target': 'cpu',
 'status': 'Starting',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '7afa5594-92cc-439d-b490-b923dc117efb',
  'azureml.git.repository_uri': 'https://github.com/danielsc/devops_issue_processing',
  'mlflow.source.git.repoURL': 'https://github.com/danielsc/devops_issue_processing',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '38f26e9abddf00d97db978e7b6019b749a8344b5',
  'mlflow.source.git.commit': '38f26e9abddf00d97db978e7b6019b749a8344b5',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'azureml-issues.py',
  'arguments': ['--data_path',
   '$AZUREML_DATAREFERENCE_b9de999b35dc420a95fcc0992e3b759e',
   '--analyze',
   '--load_open',
   '--load_closed'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu',
  'dataReferences': {'b9de999b35dc420a95fcc0992e3

In [32]:
from azureml.pipeline.core import PublishedPipeline, Schedule
old_pipes = PublishedPipeline.list(ws)

for old_pipe in old_pipes:
    old_schedules = Schedule.list(ws, pipeline_id=old_pipe.id)
    for schedule in old_schedules:
        schedule.disable(wait_for_provisioning=True)

    old_pipe.disable()

Provisioning status: Completed


In [33]:
ds = ws.get_default_datastore()

In [34]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.76


In [35]:
#ws.get_default_datastore().path('data')
params = {
    '--data_path': ws.get_default_datastore().path('data'), 
    '--analyze': '',
    '--load_open': '',
    '--load_closed': '',
}
project_folder = '.'
def_blob_store = ws.get_default_datastore()
print("Blobstore's name: {}".format(def_blob_store.name))

Blobstore's name: workspaceblobstore


In [36]:
blob_output_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="data",
    path_on_datastore="data")
print("DataReference object created")

DataReference object created


In [40]:
est = Estimator(source_directory='.', 
                compute_target=ws.compute_targets['cpu'], 
                entry_script='azureml-issues.py',
                pip_packages=['azure-devops','pandas'])

data_processing = EstimatorStep(
    estimator=est,
    estimator_entry_script_arguments=[ '--data_path', blob_output_data,
                                       '--analyze', 
                                       '--load_open', 
                                       '--load_closed'],
    inputs=[blob_output_data],
    compute_target=ws.compute_targets['cpu'],
    allow_reuse=False)
 

In [41]:
pipeline = Pipeline(workspace=ws, steps=[data_processing])
print ("Pipeline is built")

pipeline.validate()
print("Simple validation complete") 

Pipeline is built
Step azureml-issues.py is ready to be created [6dc13ca3]
Simple validation complete


In [42]:
pipeline_run = Experiment(ws, 'issues_pipeline').submit(pipeline)
print("Pipeline is submitted for execution")

Created step azureml-issues.py [6dc13ca3][5ef0ab82-168c-4bad-a2f6-de36c14fb133], (This step will run and generate new outputs)
Using data reference data for StepId [2517181a][6f26e74a-10ea-4af5-92fb-882ae2f0fa54], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 4bfc8b31-5e7d-4516-acc7-688e13c003c5
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/issues_pipeline/runs/4bfc8b31-5e7d-4516-acc7-688e13c003c5?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/issues/workspaces/issues
Pipeline is submitted for execution


In [43]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET'…

In [44]:
published_pipeline = pipeline.publish(name="Issues_Stats", description="Pull data from DevOps and aggregate for PowerBI")
print(published_pipeline.id)

576a283c-d8bf-4714-965b-e33a2f204ca8


In [47]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

recurrence = ScheduleRecurrence(frequency="Hour", interval=1, start_time="2020-01-13T12:55:00") 

schedule = Schedule.create(workspace=ws, name="Bug_Stats",
                           pipeline_id=published_pipeline.id, 
                           experiment_name='Schedule_Run',
                           recurrence=recurrence,
                           wait_for_provisioning=True,
                           description="Bug Stats Run")

# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule

print("Created schedule with id: {}".format(schedule.id))

Provisioning status: Completed
Created schedule with id: cbc4bbdb-cdd6-4671-8d46-2ed103467e94


In [51]:
schedules = Schedule.list(ws, pipeline_id=published_pipeline.id)

# We will iterate through the list of schedules and 
# use the last ID in the list for further operations: 
print("Found these schedules for the pipeline id {}:".format(published_pipeline.id))
for fetched_schedule in schedules: 
    print("Updated schedule:", fetched_schedule.id, 
          "\nNew name:", fetched_schedule.name,
          "\nNew frequency:", fetched_schedule.recurrence.frequency,
          "\nNew interval:", fetched_schedule.recurrence.interval,
          "\nNew start_time:", fetched_schedule.recurrence.start_time,
          "\nNew time_zone:", fetched_schedule.recurrence.time_zone,
          "\nNew hours:", fetched_schedule.recurrence.hours,
          "\nNew minutes:", fetched_schedule.recurrence.minutes,
          "\nNew week_days:", fetched_schedule.recurrence.week_days,
          "\nNew status:", fetched_schedule.status)



Found these schedules for the pipeline id 576a283c-d8bf-4714-965b-e33a2f204ca8:
Updated schedule: 32e6cf86-64c5-4d6e-8b5b-6e88dbfbed0a 
New name: Bug_Stats 
New frequency: Hour 
New interval: 1 
New start_time: 2020-01-13T12:45:00 
New time_zone: TimeZone.GMTStandardTime 
New hours: None 
New minutes: None 
New week_days: None 
New status: Disabled
Updated schedule: cbc4bbdb-cdd6-4671-8d46-2ed103467e94 
New name: Bug_Stats 
New frequency: Hour 
New interval: 1 
New start_time: 2020-01-13T12:55:00 
New time_zone: TimeZone.GMTStandardTime 
New hours: None 
New minutes: None 
New week_days: None 
New status: Active


In [50]:
fetched_schedule.disable()